In [1]:
import numpy as np
import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import json
from keras.callbacks import ReduceLROnPlateau

import json

batch_size = 32
num_classes = 10
epochs = 60

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

train_datagen_augm = ImageDataGenerator(
        rescale = 1/255.0,
        zoom_range = 0.1,
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1
        )

train_datagen_augm.fit(x_train, augment=True)  
validation_datagen = ImageDataGenerator(
    rescale = 1/255.0
)

Using TensorFlow backend.


In [ ]:
model_cnn_augm = keras.models.Sequential([
            keras.layers.Conv2D(32, (5,5), activation='relu', input_shape=(28, 28, 1), padding='same'),
            keras.layers.MaxPooling2D(2, 2),
            keras.layers.Dropout(0.25),
            keras.layers.Conv2D(32, (5,5), activation='relu', padding='same'),
            keras.layers.MaxPooling2D(2,2),
            keras.layers.Dropout(0.25),
            keras.layers.Flatten(),
            keras.layers.Dense(256,activation='relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10,activation='softmax')
            ])
    
model_cnn_augm.compile(
        optimizer = 'adam',
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
        )
    
print(model_cnn_augm.summary())

lrate = ReduceLROnPlateau(monitor='val_accuracy',
                      factor=0.4,
                      patience=3,
                      verbose=1,
                      min_lr=0.0001)

callbacks_list = [lrate]

training_set = train_datagen_augm.flow(x_train, y_train, batch_size=batch_size)
validation_set = validation_datagen.flow(x_test, y_test, batch_size=batch_size)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 32)          0         
______________________________________

In [ ]:
history_augm = model_cnn_augm.fit_generator(
        training_set,
        #steps_per_epoch=training_set.n // training_set.batch_size,
        epochs=epochs,
        verbose=1,
        #callbacks=callbacks_list,
        #validation_data=validation_set,
        #validation_steps=validation_set.n // validation_set.batch_size)
)


Epoch 1/60
